### COP Data Files - processing
This file is build to open and process data files recored by force platform constructed by the company AMTI. It is used in both sport and clinical studies to measure body posture. To achieve this goal it records with a high frequency forces, which can be used to calculate body's Center Of Pressure (COP), and it's translations give some insight into postural stability of a subject. 

In [18]:
import pandas as pd
import numpy as np
from scipy import signal
from pyentrp import entropy as ent
import glob
import chardet

## 1) Import of selected data

All data recorded form force platform is stored in .txt files. Each contain 6 columns with forces in frontal, medial and vertical plane and also moments of forces in same directions. Data is usually recored with the frequency of 50-100Hz, and for 30 seconds which gives 1500-3000 rows of data in this example. Plaese use the correct pattern file depending on your operating system!

In [19]:
# Pattern for folder, and file type selection (Linux/Macos)
#! On Windows you need to replace all '/' with '\\'
patternFiles = 'Data/COP_Data/Group_1/*.txt'

# Initial filename
patternFilename = 'Data/COP_Data/Group_1/'

# Resulting xls file name
patternExport = 'DataBase/COP_DB_Group_1.xlsx'

# Gather all files
files = glob.glob(patternFiles)

Chardet is used here only to verify file encoding. Its not necessary to uncomment it in order to run rest of the code

In [20]:
# with open(files[0], 'rb') as file:
#     result = chardet.detect(file.read())
#     print(result)
# df= pd.read_csv(files[0], skiprows=5, header = None, encoding="Windows-1252")

Class for text attributes (Important for aesthethics of print outs)

In [21]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## 2) Calculating essential data in each file from chosen path

First each file will be filtered with 4th order Butterworth filter with cutoff frequency set to 7hz.
After this for each measurement there will be calculated COP (Center of Pressure) for medial and frontal plane with all characteristics describing its movement (velocity, range of displacement, lentgh). Last parameter in this part of analysis will be calculating Sample Entropy of each measurement. 

In [22]:
#Initializing empty DataFrame to store results for each measurement
COP_Result = pd.DataFrame(columns= ['File', 'raCOP_AP', 'stdCOP_AP', 'rmsCOP_AP', 'lenCOP_AP', 'vCOP_AP', 
                                    'sampEN_AP', 'std_sampeEN_AP','raCOP_ML', 'stdCOP_ML', 'rmsCOP_ML', 
                                    'lenCOP_ML', 'vCOP_ML', 'sampEN_ML', 'std_sampeEN_ML'])

#Counter for files (only for information about progress)
fileCount = 1
fileAmount = len(files)

In [23]:
# Looping through files stored in selected folder
for file in files:
    
    #Read file from file list
    df= pd.read_csv(file, header = None, encoding="Windows-1252", skiprows=4)
    df.columns = ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']
    
    #Filename for a file (cutting path to folder so only file name stays)
    fileName = file.replace(patternFilename, '')
    
    #Filtering data with Butterworth filter
    
    #Cutoff frequency
    fc = 7

    #Sampling frequency used in measurement
    fs = 50

    #Filter definition
    b, a = signal.butter(4, fc/(fs/2), 'low', analog=False)

    df['Fx'] = signal.filtfilt(b,a, df['Fx'])
    df['Fy'] = signal.filtfilt(b,a, df['Fy'])
    df['Fz'] = signal.filtfilt(b,a, df['Fz'])
    df['Mx'] = signal.filtfilt(b,a, df['Mx'])
    df['My'] = signal.filtfilt(b,a, df['My'])
    df['Mz'] = signal.filtfilt(b,a, df['Mz'])
    
    
    #Assigning timeline for data
    df['Time'] = [ind / fs for ind in df.index]
    
    
    #Calculation of center of pressure for both directions - frontal and medial
    df['COPX'] = [ -(My/Fz) *100 for My,Fz in zip(df['My'], df['Fz']) ]
    df['COPY'] = [ -(-Mx/Fz) * 100 for Mx,Fz in zip(df['Mx'], df['Fz'])]
    

    #Calculating COP characteristics in anterior-posterior direction (AP)
    raCOP_AP = df['COPY'].max() - df['COPY'].min()
    stdCOP_AP = df['COPY'].std()
    rmsCOP_AP = np.sqrt( df['COPY'].mean() **2 )   
    lenCOP_AP = 0

    for i in range(0, len(df['COPY'])-1) :
        lenCOP_AP += abs(df['COPY'][i+1] - (df['COPY'][i]) ) 

    vCOP_AP = lenCOP_AP / ( len(df['COPY']) / fs ) 
    sampeEn_AP = ent.sample_entropy(df['COPY'], 3, 0.2)

    #Calculating COP characteristics in medio-lateral plane (ML)
    raCOP_ML = df['COPX'].max() - df['COPX'].min()
    stdCOP_ML = df['COPX'].std()
    rmsCOP_ML = np.sqrt( df['COPX'].mean() **2)   
    lenCOP_ML = 0

    for i in range(0, len(df['COPX'])-1) :
        lenCOP_ML += abs(df['COPX'][i+1] - (df['COPX'][i]) ) 

    vCOP_ML = lenCOP_ML / ( len(df['COPX']) / fs ) 
    sampeEN_ML = ent.sample_entropy(df['COPX'], 3, 0.2)

    #Create a new df with a result
    rowData = pd.DataFrame({'File': [fileName], 'raCOP_AP': [raCOP_AP], 'stdCOP_AP': [stdCOP_AP], 'rmsCOP_AP': [rmsCOP_AP], 
                                    'lenCOP_AP': [lenCOP_AP], 'vCOP_AP' : [vCOP_AP], 'sampEN_AP' : [sampeEn_AP[0]], 
                                    'std_sampeEN_AP': [sampeEn_AP[1]], 'raCOP_ML': [raCOP_ML], 'stdCOP_ML': [stdCOP_ML], 
                                    'rmsCOP_ML': rmsCOP_ML, 'lenCOP_ML': lenCOP_ML, 'vCOP_ML': vCOP_ML, 
                                    'sampEN_ML': [sampeEN_ML[0]], 'std_sampeEN_ML': [sampeEN_ML[1]] })
    
    # Add result to the main data frame
    COP_Result = pd.concat([COP_Result, rowData], ignore_index=True)
    
    #Information about progress
    print(color.BOLD + 'Total progress : ' + color.END + str(int(fileCount/fileAmount  *100)) + '%', end = '\r')
    fileCount = fileCount + 1
    

#Print a message with short summary after analysis is done    
print('\n' + '\n' + color.BOLD + color.GREEN + 'All data processed succesfully !!' + color.END + '\n' +  color.BOLD + 'Amount of files processd = ' + color.END + str(len(files)))

/tmp/ipykernel_13402/2122209257.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  COP_Result = pd.concat([COP_Result, rowData], ignore_index=True)


Total progress : 100%

All data processed succesfully !!
Amount of files processd = 45


## Export result dataframe into excel file
Database that will be used in the 2nd part of the analysis consists of all 3 groups data combined, with a category column added. Also to make this final database I used more data. This was done manually therefore I the file is already in database folder. Data processed here is not required for the next steps

In [24]:
COP_Result.sort_values(by="File", ascending=True, inplace=True)
COP_Result.to_excel(patternExport, index=False)

1. Dodaj README
2. Push